In [1]:
import akshare as ak
import pandas as pd

In [2]:
ag9999 = ak.spot_hist_sge(symbol='Ag99.99')
agm = ak.futures_hist_em(symbol="沪银主连", period="daily")

In [3]:
df = pd.merge(ag9999, agm, left_on='date', right_on='时间', how='left')
df = df.rename(columns = {'close': 'ag9999', '收盘': 'agm'})
df = df[['date', 'ag9999', 'agm']]
df['date'] = pd.to_datetime(df['date'])
df['premium'] = df['ag9999'] - df['agm']
print(df['premium'].abs().describe())
print(df['premium'].abs().quantile(0.99))

count    702.000000
mean      40.374644
std       53.530458
min        0.000000
25%       12.000000
50%       24.000000
75%       53.000000
max      802.000000
Name: premium, dtype: float64

In [6]:
def calculate_drawdown_time(row, val):
     current_date = row['date']
     current_premium_abs = abs(row['premium'])
     if current_premium_abs <= val:
         return pd.Timedelta(0)
     subsequent_data = df[df['date'] > current_date]
     drawdown_index = subsequent_data[subsequent_data['premium'].abs() <= val].first_valid_index()
     if drawdown_index is not None:
         drawdown_date = df.loc[drawdown_index, 'date']
         return drawdown_date - current_date
     return pd.NaT
sell_point = 24

df[f'down_{sell_point}'] = df.apply(calculate_drawdown_time, args=(sell_point,), axis=1)

In [15]:
threshold = 110
signal_df = df[(df['premium'] > threshold) | (df['premium'] < -1 * threshold)]
print(df['premium'].abs().quantile(0.95))
print(len(signal_df) * 100 / len(df))

signal_df

108.94999999999993
4.415954415954416


,date,ag9999,agm,premium,down_24
0,2016-12-19,3999.0,4116,-117.0,128 days
6,2016-12-29,3968.0,4087,-119.0,118 days
8,2017-01-04,4016.0,4130,-114.0,112 days
11,2017-01-09,3995.0,4109,-114.0,107 days
14,2017-01-12,4038.0,4150,-112.0,104 days
48,2017-05-10,3886.0,4002,-116.0,33 days
49,2017-05-12,3942.0,4053,-111.0,31 days
50,2017-05-26,3998.0,4125,-127.0,17 days
52,2017-06-06,4040.0,4164,-124.0,6 days
54,2017-06-08,4052.0,4163,-111.0,4 days


In [16]:
df[625:665]

,date,ag9999,agm,premium,down_24
638,2024-07-05,8093.0,8086,7.0,0 days
639,2024-07-08,8243.0,8126,117.0,16 days
640,2024-07-22,7630.0,7602,28.0,2 days
641,2024-07-23,7579.0,7536,43.0,1 days
642,2024-07-24,7668.0,7688,-20.0,0 days
643,2024-07-29,7315.0,7337,-22.0,0 days
644,2024-08-05,7231.0,7249,-18.0,0 days
645,2024-08-06,7038.0,6907,131.0,2 days
646,2024-08-08,6933.0,6929,4.0,0 days
647,2024-08-21,7544.0,7562,-18.0,0 days
